In [ ]:
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
from PIL import Image

import matplotlib.gridspec as gridspec
import tensorflow as tf
import numpy as np

tf.executing_eagerly()

True

In [ ]:
def get_noise_image(content_img, seed):
    np.random.seed(seed)
    img = np.random.uniform(0, 1, content_img.shape).astype(np.float32)
    return img


def load_and_process_image(content_path, style_path):
    content_img = Image.open(content_path)
    content_img = np.array(content_img)
    content_shape = content_img.shape[:2]
    content_shape = content_shape[::-1]
    
    style_img = Image.open(style_path)
    style_img = style_img.resize(content_shape)
    style_img = np.array(style_img)

    content_img = tf.expand_dims(content_img, axis=0)
    style_img = tf.expand_dims(style_img, axis=0)
    
    return content_img, style_img

# ???
def deprocess_image(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
  
    # perform the inverse of the preprocessing step
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x


def display_result(content_path, style_path, best_img=None):
    content_img, style_img = load_and_process_image(content_path, style_path)

    if len(content_img.shape) > 3:
      content_img = np.squeeze(content_img, axis=0)

    if len(style_img.shape) > 3:
      style_img = np.squeeze(style_img, axis=0)
    
    fig = plt.figure(figsize=(13,13))
    gridspec.GridSpec(2,2)

    plt.subplot2grid((3,2), (0,0))
    plt.imshow(content_img)
    plt.title('Content Image')
    plt.axis('off')
    
    plt.subplot2grid((3,2), (0,1))
    plt.imshow(style_img)
    plt.title('Style Image')
    plt.axis('off')
    
    if best_img is not None:
        plt.subplot2grid((3,2), (1,0), colspan=2, rowspan=2)
        plt.imshow(best_img)
        plt.title('Style Transfer Output')
        plt.axis('off')

    fig.tight_layout()


In [ ]:
def build_model():
    
    # VGG19 based 
    vgg19 = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg19.trainable = False
    
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    content_outputs = [vgg19.get_layer(name).output for name in content_layers]
    style_outputs = [vgg19.get_layer(name).output for name in style_layers]

    model_outputs = content_outputs + style_outputs

    model = Model(inputs=vgg19.input, outputs=model_outputs)
    
    return model
    

def feature_extract(model, img, mode, num_content_layers=1):
    outputs = model(img)

    if mode == 'content':
      feature = [out for out in outputs[:num_content_layers]]
    elif mode == 'style':
      feature = [out for out in outputs[num_content_layers:]]
    
    return feature


In [ ]:
def get_content_loss(base_content, target_content):
    return tf.reduce_mean(tf.square(base_content - target_content))


def get_style_loss(base_style, target_style):
    return tf.reduce_mean(tf.square(base_style - target_style))
    

def compute_loss(model, base_img, target_content, target_style, content_weight, style_weight):
    
    content_loss = 0
    style_loss = 0
    
    base_content = feature_extract(model, base_img, mode='content')
    base_style = feature_extract(model, base_img, mode='style')
    
    layer_weight_content = 1/len(target_content)
    for bc, tc in zip(base_content, target_content):
        content_loss += layer_weight_content * get_content_loss(bc, tc)
        
    layer_weight_style = 1/len(target_style)
    for bs, ts in zip(base_style, target_style):
        style_loss += layer_weight_style * get_style_loss(bs, ts)
        
    content_loss *= content_weight
    style_loss *= style_weight
    
    total_loss = content_loss + style_loss
    
    return total_loss, content_loss, style_loss
        

In [ ]:
def compute_grad(config):
    with tf.GradientTape() as tape:
        tape.watch(config['base_img'])
        total_loss = compute_loss(**config)[0]
    return tape.gradient(total_loss, config['base_img']), total_loss


In [ ]:
def style_transfer(content_path, style_path, content_weight, style_weight, num_iterations=500):
    model = build_model()

    content_img, style_img = load_and_process_image(content_path, style_path)
    
    content_img = tf.keras.applications.vgg19.preprocess_input(content_img)
    style_img = tf.keras.applications.vgg19.preprocess_input(style_img)

    base_img = tf.identity(content_img)
    base_img = tf.cast(base_img, dtype='float')
    base_img = tf.Variable(base_img)

    noise_img = get_noise_image(content_img, 0)
    base_img = tf.Variable(noise_img.copy())
    
    target_content = feature_extract(model, content_img, mode='content')
    target_style = feature_extract(model, style_img, mode='style')
    
    config = {'model': model,
              'base_img': base_img,
              'target_content': target_content,
              'target_style': target_style,
              'content_weight': content_weight,
              'style_weight': style_weight}
    
    norm_means = np.array([103.939, 116.779, 123.68])
    min_vals = -norm_means
    max_vals = 255 - norm_means 
    
    best_loss, best_image = float('inf'), None
    
    optim = tf.keras.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=0.1)

    total_loss_hist = np.zeros(num_iterations)
    
    for i in range(num_iterations):
        grad, total_loss = compute_grad(config)
        total_loss_hist[i] = total_loss
        optim.apply_gradients([(grad, base_img)])
        # ???
        clipped_img = tf.clip_by_value(base_img, min_vals, max_vals)
        base_img.assign(clipped_img)
        
        if total_loss < best_loss:
            best_loss = total_loss
            best_img = deprocess_image(base_img.numpy())
        
    return best_loss, best_img, total_loss_hist


In [ ]:
content_path = 'Image/content1.jpg'
style_path = 'Image/style1.jpg'

In [ ]:
content_weight = 1e3
style_weight = 1e-2
num_iterations = 1000

best_loss, best_img, total_loss_hist = style_transfer(content_path, style_path, content_weight, style_weight, num_iterations=num_iterations)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plt.style.use('ggplot')

ax.plot(range(1,num_iterations), total_loss_hist[1:], alpha=0.6, label='Total Loss')
plt.xlabel('Number of Iterations')
plt.ylabel('Loss')
plt.title('Loss vs Number of Iterations')
plt.legend()

plt.show()

In [ ]:
display_result(content_path, style_path, best_img)

Output hidden; open in https://colab.research.google.com to view.